In [1]:
#安装第三方包
pip install pandas openpyxl numpy pyperclip

Note: you may need to restart the kernel to use updated packages.


In [1]:
# 导入
import pandas as pd
from datetime import datetime
import re
from openpyxl.utils import get_column_letter
from pandas import ExcelWriter
import numpy as np
import pyperclip
import os

In [6]:
# 输入上次和本次采集数据的日期
datetime_last = datetime(2025,7,13) # 上次采集数据的日期
datetime_this = datetime(2025,7,14) # 本次采集数据的日期

# 构造文件路径
def get_file_paths(date):
    date_str = date.strftime('%Y-%m-%d')
    raw_path = f'raw_data/calculate-{date_str}.xlsx'
    save_path = f'raw_data/proceed/calculate-{date_str}.xlsx'
    return raw_path, save_path

# 创建保存目录（如果不存在）
os.makedirs('raw_data/proceed', exist_ok=True)

# 处理单个文件
def process_file(input_path, output_path):
    # 读取文件
    df = pd.read_excel(input_path)

    # 2.1 去除前两列
    df = df.iloc[:, 2:]

    # 2.2 去除第三列中的所有“#”，并转换为数值型以便正确排序
    if df.shape[1] >= 1:
        df.iloc[:, 0] = df.iloc[:, 0].astype(str).str.replace('#', '', regex=False)
        df.iloc[:, 0] = pd.to_numeric(df.iloc[:, 0], errors='coerce')  # 转成数字，非法项为NaN

    # 2.3 根据第三列排序（现在是第一列）
    df = df.sort_values(by=df.columns[0], ascending=True)

    # 2.4 去除重复行
    df = df.drop_duplicates()
    
    # 2.5 将最后一列中的空值替换为 0
    if df.shape[1] >= 1:
        df.iloc[:, -1] = df.iloc[:, -1].fillna(0)
    
    # 保存结果
    df.to_excel(output_path, index=False)

# 处理两个文件
for dt in [datetime_last, datetime_this]:
    input_file, output_file = get_file_paths(dt)
    process_file(input_file, output_file)

print("文件处理完毕。")


C:\Users\19872\AppData\Local\Temp\ipykernel_31020\3866836531.py:26: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  df.iloc[:, 0] = pd.to_numeric(df.iloc[:, 0], errors='coerce')  # 转成数字，非法项为NaN


文件处理完毕。


C:\Users\19872\AppData\Local\Temp\ipykernel_31020\3866836531.py:26: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  df.iloc[:, 0] = pd.to_numeric(df.iloc[:, 0], errors='coerce')  # 转成数字，非法项为NaN


In [19]:
import pandas as pd
from datetime import datetime

# 设置用户名
username = '斯格里芬'

# 构造文件路径
def get_file_path(date):
    date_str = date.strftime('%Y-%m-%d')
    return f'raw_data/proceed/calculate-{date_str}.xlsx'

# 读取两个文件
df_last = pd.read_excel(get_file_path(datetime_last))
df_this = pd.read_excel(get_file_path(datetime_this))

# 按用户名是否相同进行分组
def split_by_username(df, username_column='author'):
    if username_column not in df.columns:
        raise ValueError(f"列 '{username_column}' 不存在，请检查 Excel 文件。")
    return df[df[username_column] == username], df[df[username_column] != username]

df_last_same, df_last_diff = split_by_username(df_last)
df_this_same, df_this_diff = split_by_username(df_this)

# 生成当天帖子的统计报告（分析 df_this）
df_total = df_this
df_same = df_this_same
df_diff = df_this_diff

total_rows = len(df_total)
same_rows = len(df_same)
diff_rows = len(df_diff)
total_recommend = df_total.iloc[:, -1].sum()
same_recommend = df_same.iloc[:, -1].sum()
diff_recommend = df_diff.iloc[:, -1].sum()

# 输出总览
print("===== 分析报告 =====")
print(f"在 {datetime_this.date()} 的帖子中")
print(f"一共有 {total_rows} 条回复。")
print(f"其中楼主更新了 {same_rows} 条，")
print(f"观众回复了 {diff_rows} 条，")
print(f"总推荐数为 {total_recommend}，")
print(f"楼主获得推荐为 {same_recommend}，")
print(f"观众回复获得推荐为 {diff_recommend}")

# 差异分析函数
def analyze_diff(df1, df2, label, is_reply=False):
    rows_1 = len(df1)
    rows_2 = len(df2)
    row_diff = rows_2 - rows_1

    sum_1 = df1.iloc[:, -1].sum()
    sum_2 = df2.iloc[:, -1].sum()
    sum_diff = sum_2 - sum_1

    prefix = "[回复]" if is_reply else "[更新]"
    print(f"{prefix} 增加了 {row_diff}{'回复' if is_reply else '次'}。")

    prefix_reco = "[回复推荐]" if is_reply else "[推荐]"
    print(f"{prefix_reco} 回复增加了 {sum_diff} 推荐。" if is_reply else f"[推荐] 增加了 {sum_diff} 推荐。")

    # 查找推荐变化的行
    if df1.empty or df2.empty:
        print("[推荐增加] 无可比较内容。")
        return

    cols_except_last = df1.columns[:-1]
    df1_indexed = df1.set_index(list(cols_except_last))
    df2_indexed = df2.set_index(list(cols_except_last))

    common_index = df1_indexed.index.intersection(df2_indexed.index)
    if common_index.empty:
        print("[推荐增加] 无完全相同的行。")
        return

    last_col_1 = df1_indexed.loc[common_index].iloc[:, -1]
    last_col_2 = df2_indexed.loc[common_index].iloc[:, -1]

    changed = last_col_1 != last_col_2
    changed_rows = common_index[changed]

    if not changed_rows.empty:
        label_text = "回复" if is_reply else "楼主楼层"
        print(f"[推荐增加] 发生推荐变化的{label_text}：")
        for idx in changed_rows:
            try:
                floor = idx[0]  # 假设第一列是楼层号
                print(f"  第 {floor} 楼增加了 {last_col_2.loc[idx] - last_col_1.loc[idx]} 推荐")
            except Exception as e:
                print(f"  [错误行] {list(idx)}: {e}")
    else:
        print(f"[推荐增加] {datetime_last.date()}当天所有{'回复' if is_reply else '楼层'}都没有推荐增加")

# 输出变化部分
print(f"\n相比于 {datetime_last.date()}：")
analyze_diff(df_last_same, df_this_same, label="楼主", is_reply=False)
analyze_diff(df_last_diff, df_this_diff, label="回复", is_reply=True)


===== 分析报告 =====
在 2025-07-14 的帖子中
一共有 315 条回复。
其中楼主更新了 232 条，
观众回复了 83 条，
总推荐数为 802，
楼主获得推荐为 713，
观众回复获得推荐为 89

相比于 2025-07-13：
[更新] 增加了 3次。
[推荐] 增加了 8 推荐。
[推荐增加] 发生推荐变化的楼主楼层：
  第 305 楼增加了 1 推荐
  第 306 楼增加了 1 推荐
  第 307 楼增加了 1 推荐
  第 308 楼增加了 1 推荐
  第 310 楼增加了 1 推荐
[回复] 增加了 1回复。
[回复推荐] 回复增加了 1 推荐。
[推荐增加] 2025-07-13当天所有回复都没有推荐增加
